In [ ]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns

from bmp_config import path_data, envcode2env
from bmp_behav_proc import *

fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
print(df_all_multi_tsz['trial_group_col_calc'].unique())
print(df_all_multi_tsz['retention_factor_s'].unique())
df_all_multi_tsz_orig = df_all_multi_tsz

In [ ]:
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_orig = df
# define dfall here
df,dfall,ES_thr,envv,pert = addBehavCols2(df);


In [ ]:
assert df_orig.groupby('subject').size().min() == 768

In [ ]:
print( df_all_multi_tsz['trial_group_col_calc'].unique() )
checkErrBounds(df,['error','prev_error','error_deg'])
assert dfall.query('env == "stable"').trialwpertstage_wb.max() < 60

In [ ]:
#print( dfall['subject_ind'].unique() )
dfall_os = dfall.query('subject_ind == 1').copy()
assert len(dfall_os) > 600, len(dfall_os)

In [ ]:
from figure.plots import Fig2_annotate_segments
tr = np.arange(len(pert))
c = 1 / np.pi  * 180
svg_files = []
qs = None
for lablet,varn,pertc,figsz,y_tl,y_title in [('A', 'error_pscadj', 1.,(12,10),1.15, 1.48), 
                   ('B', 'err_sens', 0.06,(12,4),1.,1.)]:

    varn_eff = varn
    if varn.startswith('org_feedback'):
        dfall_os['vtp'] = np.array(dfall_os[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        dfall_os['vtp'] = np.array(dfall_os[varn]) * c 
        varn_eff = 'vtp'

    plt.figure(figsize=figsz)
    fg = sns.relplot(data=dfall_os,
        kind='line', x="trials", y=varn_eff, 
         estimator='mean', errorbar='sd', height = 4, aspect=2.5)

    if varn == 'error_pscadj':
        Fig2_annotate_segments(fg.ax, dfall_os, qs, category_col='ps2_', rotation_angle=60,
            ps_2nice = ps_2nice, text_size = 12, text_y_position = 35, x_shift=-8)
    else:
        Fig2_annotate_segments(fg.ax, dfall_os, qs, category_col='ps2_', 
            ps_2nice = ps_2nice, show_text = False)


    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--', lw=4)
        #ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn )
    if varn.startswith('error_'):
        fg.set_ylabels('Angular error [deg]')
        fg.set_titles('Error dynamics', y =y_title)
    else:
        fg.set_ylabels('Error sensitivity')
        fg.set_titles('Error sensitivity dynamics')
    fg.set_xlabels('Trial number')
 
    #addTitleInfo(fg.axes.flatten()[0])
    # ax.annotate(lablet, xy=(0, y_title), xytext=(-60, 20), 
    #   fontsize=19, fontweight='bold', va='top', ha='left',
    #   xycoords='axes fraction', textcoords='offset points')
    ax.axhline(0, color='black', lw=1, ls='--')

    fign = 'Fig2_os'+lablet
    fnfbase = pjoin(path_fig, 'behav',fign )
    fnfsvg = fnfbase + '.svg'
    #plt.tight_layout(rect=[0,0,1,y_tl])
    plt.tight_layout()
    plt.savefig(fnfsvg, bbox_inches='tight')
    plt.savefig(fnfbase + '.pdf')
    plt.close()

    svg_files += [fnfsvg]

from figure.imgfilemanip import *
#svg_files = [pjoin(path_fig, 'behav','Fig2'+lablet + '.svg') for lablet in ['A','B'] ]
print(svg_files)
fnfout = pjoin(path_fig, 'behav',pjoin(path_fig, 'behav','Fig2AB' + '.svg'))
stackSVGandShowJupy(svg_files, 'vertical', fnfout)

In [ ]:
display(svg_files)
svg2png(svg_files[0], dpi=300)